# DK-SAP （Plain）

In [ ]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
from eth_account import Account
from ecdsa import SigningKey, SECP256k1 
import time

In [ ]:
def sk_PK_HPK_WA(sk):
    # PK = secp256k1.privtopub(s.to_bytes(32, "big"))
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))
    
    HPK = sha3.keccak_256(PK[0].to_bytes(32, "big")+PK[1].to_bytes(32, "big")).hexdigest()
    WA = "0x"+ HPK[-40:]
    return PK, HPK, WA
    
sk_george = int(0x13803677559c049a98a1576ad2f1fd15bdd9f81886f225c0850e883bd7d4cf63)
PK_george, HPK_george, WA_george = sk_PK_HPK_WA(sk_george)

name = 'george'
print(name)
print('==== G: \n', secp256k1.G)
print('==== sk: \n', sk_george)
print('==== PK=G*sk \n', PK_george)
print('==== HPK=h256(PK) \n', HPK_george)
print('==== WA=-40(HPK): \n', WA_george)


def get_PK(sk):
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))
    return PK

def sa_PK_scan_spend(sk_scan, sk_spend):
    PK_scan, HPK_scan, WA_scan = sk_PK_HPK_WA(sk_scan)
    PK_spend, HPK_spend, WA_spend = sk_PK_HPK_WA(sk_spend)
    return PK_scan, PK_spend

def register_PKscan_PKspend(PK_scan, PK_spend):
    pass

def gen_SA(r, PK_scan, PK_spend):
    R = get_PK(r)
    
    ss_alice = secp256k1.multiply(PK_scan, r)
    #print('Alice gen ss: \n', ss_alice)

    hss_alice = sha3.keccak_256(ss_alice[0].to_bytes(32, "big") + ss_alice[1].to_bytes(32, "big")).hexdigest()
    hss_alice = bytearray.fromhex(hss_alice)
    #print('Alice gen h256(ss): \n', hss_alice)

    PK_sa_alice = secp256k1.add(PK_spend, secp256k1.privtopub(hss_alice))
    #print('Alice gen PK_sa: \n', PK_sa_alice)

    HPK_sa_alice = sha3.keccak_256(PK_sa_alice[0].to_bytes(32, "big")
                        +PK_sa_alice[1].to_bytes(32, "big")
                        ).hexdigest()
    #print('Alice gen HPK_sa: \n', HPK_sa_alice)

    WA_sa_alice = "0x"+ HPK_sa_alice[-40:]
    #print('Alice gen WA_sa: \n', WA_sa_alice)
    
    return R, WA_sa_alice

def scan_SA(R, sk_scan, PK_spend):
    ss_bob = secp256k1.multiply(R, sk_scan)

    hss_bob = sha3.keccak_256(ss_bob[0].to_bytes(32, "big") + ss_bob[1].to_bytes(32, "big")).hexdigest()
    hss_bob = bytearray.fromhex(hss_bob)
    #print('Bob gen h256(ss): \n', hss_bob)
    
    PK_sa_bob = secp256k1.add(PK_spend, secp256k1.privtopub(hss_bob))
    #print('Alice gen PK_sa: \n', PK_sa_alice)

    HPK_sa_bob = sha3.keccak_256(PK_sa_bob[0].to_bytes(32, "big")
                            +PK_sa_bob[1].to_bytes(32, "big")
                            ).hexdigest()
    #print('Bob gen HPK_sa: \n', HPK_sa_alice)

    WA_sa_bob = "0x"+ HPK_sa_bob[-40:]
    #print('Bob gen WA_sa: \n', WA_sa_bob)
    
    return WA_sa_bob

def spend_SA(R, sk_scan, sk_spend):
    ss_bob = secp256k1.multiply(R, sk_scan)

    Hss_bob = sha3.keccak_256(ss_bob[0].to_bytes(32, "big") + ss_bob[1].to_bytes(32, "big")).hexdigest()
    #print('Bob gen h256(ss): \n', Hss_bob)

    sk_sa_bob = sk_spend + int(Hss_bob, 16)
    #print("Bob gen sk_sa: \n", sk_sa_bob)

    PK_sa_bob = secp256k1.privtopub(sk_sa_bob.to_bytes(32, "big"))
    #print('Bob gen PK_sa: \n', PK_sa_bob)

    HPK_sa_bob = sha3.keccak_256(PK_sa_bob[0].to_bytes(32, "big")
                            +PK_sa_bob[1].to_bytes(32, "big")
                            ).hexdigest()
    #print('Bob gen HPK_sa: \n', HPK_sa_bob)

    WA_sa_bob = "0x"+ HPK_sa_bob[-40:]
    #print('Bob gen WA_sa: \n', WA_sa_bob)
    
    return sk_sa_bob, WA_sa_bob

    
def register_R(R):
    pass

def send_to_SA(WA_SA):
    pass

In [ ]:
for i in range(100):
    sum = 0
    start = time.perf_counter()
    sk_scan = int(0x0000000000000000000000000000000000000000000000000000000000000002) # private key
    sk_spend = int(0x0000000000000000000000000000000000000000000000000000000000000003) # private key
    PK_scan, PK_spend = sa_PK_scan_spend(sk_scan, sk_spend)
    register_PKscan_PKspend(PK_scan, PK_spend)
    r = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30) # private key
    R, SA_alice = gen_SA(r, PK_scan, PK_spend)
    register_R(R)
    send_to_SA(SA_alice)
    SA_scan = scan_SA(R, sk_scan, PK_spend)
    SA_scan.lower() == SA_alice.lower()
    sk_SA_spend, SA_spend = spend_SA(R, sk_scan, sk_spend)
    SA_to_spend = Account.from_key((sk_SA_spend).to_bytes(32, "big")).address
    SA_to_spend.upper() == SA_spend.upper()
    end = time.perf_counter()
    elapsed_time = end - start
    int(elapsed_time)
    print(elapsed_time)
    i+1


# DK-SAP Paillier

In [ ]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
from eth_account import Account
from ecdsa import SigningKey, SECP256k1 
from phe import paillier
import time
import timeit

In [ ]:
def sk_PK_HPK_WA(sk):
    # PK = secp256k1.privtopub(s.to_bytes(32, "big"))
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))  
    HPK = sha3.keccak_256(PK[0].to_bytes(32, "big")+PK[1].to_bytes(32, "big")).hexdigest()
    WA = "0x"+ HPK[-40:]
    return PK, HPK, WA
    
sk_bob = int(0x13803677559c049a98a1576ad2f1fd15bdd9f81886f225c0850e883bd7d4cf63)
PK_bob, HPK_bob, WA_bob = sk_PK_HPK_WA(sk_bob)

name = 'Bob'
print(name)
#print('==== G: \n', secp256k1.G)
print('==== sk_bob: \n', sk_bob)
print('==== PK_bob=G*sk_bob \n', PK_bob)
#print('==== HPK=h256(PK) \n', HPK_bob)
print('==== WA2=-40(HPK): \n', WA_bob)


def get_PK(sk):
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))
    return PK

def sk_PK_HPK_WA(sk):
    # PK = secp256k1.privtopub(s.to_bytes(32, "big"))
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))  
    HPK = sha3.keccak_256(PK[0].to_bytes(32, "big")+PK[1].to_bytes(32, "big")).hexdigest()
    WA = "0x"+ HPK[-40:]
    return PK, HPK, WA
    
sk_bob = int(0x13803677559c049a98a1576ad2f1fd15bdd9f81886f225c0850e883bd7d4cf63)
PK_bob, HPK_bob, WA_bob = sk_PK_HPK_WA(sk_bob)

name = 'Bob'
print(name)
#print('==== G: \n', secp256k1.G)
print('==== sk_bob: \n', sk_bob)
print('==== PK_bob=G*sk_bob \n', PK_bob)
#print('==== HPK=h256(PK) \n', HPK_bob)
print('==== WA2=-40(HPK): \n', WA_bob)


def get_PK(sk):
    PK = secp256k1.multiply(secp256k1.G, secp256k1.bytes_to_int(sk.to_bytes(32, "big")))
    return PK

In [ ]:
for i in range(100):
    sum = 0
    start = time.perf_counter()
    public_key, private_key = paillier.generate_paillier_keypair()
    C2 = public_key.encrypt(sk_bob)
    sk_alice_fhe = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30)
    PK_alice_fhe = get_PK(sk_alice_fhe)
    PK_fhe_alice = secp256k1.add(PK_alice_fhe, PK_bob)
    HPK_fhe_alice = sha3.keccak_256(PK_fhe_alice[0].to_bytes(32,'big')+PK_fhe_alice[1].to_bytes(32,'big')).hexdigest()
    WA_fhe_alice = "0x"+ HPK_fhe_alice[-40:]
    C1 = public_key.encrypt(sk_alice_fhe)
    C = C1 + C2
    sk_fhe = private_key.decrypt(C)
    PK_fhe_bob = get_PK(sk_fhe)
    HPK_fhe_bob = sha3.keccak_256(PK_fhe_bob[0].to_bytes(32,'big')+PK_fhe_bob[1].to_bytes(32,'big')).hexdigest()
    WA_fhe_bob = "0x"+ HPK_fhe_bob[-40:]
    end = time.perf_counter()
    elapsed_time = end - start
    print(elapsed_time)
    i+1


In [ ]:

!pip install py_ecc
!pip uninstall sha3
!pip install ecdsa
!pip install pysha3
!pip install eclib


# DP-SAP (concrete)

In [ ]:
import hashlib
from py_ecc.secp256k1 import *
import sha3
from eth_account import Account
from ecdsa import SigningKey, SECP256k1 
import time
from concrete import fhe

In [ ]:
import time
from concrete import fhe

start = time.perf_counter()
def add(x, y):
    return x + y

compiler = fhe.Compiler(add, {"x": "encrypted", "y": "encrypted"})
inputset = [(2**56, 0x7d004cf63)]

print(f"Compiling...")
circuit = compiler.compile(inputset)

print(f"Generating keys...")
circuit.keygen()

def split(a):
    return [int(str(a)[::-1][i*16:(i+1)*16][::-1]) for i in range(-(-len(str(a))//16))]
def bind_split(a,b):
    return list(zip(a+[0]*(max(len(a), len(b))-len(a)), b+[0]*(max(len(a), len(b))-len(b))))
def combine(l):
    return sum([l[i]*(10**(16*i)) for i in range(len(l))])

a = 0x13803677559c049a98a1576ad2f1fd15bdd9f81886f225c0850e883bd7d4cf63
b = 0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30


examples = bind_split(split(a), split(b))
result_list = []
for example in examples:
    encrypted_example = circuit.encrypt(*example)
    encrypted_result = circuit.run(encrypted_example)
    result = circuit.decrypt(encrypted_result)
    result_list.append(result)
    print(f"Evaluation of {' + '.join(map(str, example))} homomorphically = {result}")
combine(result_list)

In [ ]:
for i in range(100):
    sum = 0
    start = time.perf_counter()
    
    def add(x, y):
    return x + y

    compiler = fhe.Compiler(add, {"x": "encrypted", "y": "encrypted"})
    inputset = [(2**56, 0x7d004cf63)]

    print(f"Compiling...")
    circuit = compiler.compile(inputset)

    print(f"Generating keys...")
    circuit.keygen()
    
    sk_alice_fhe = int(0xd952fe0740d9d14011fc8ead3ab7de3c739d3aa93ce9254c10b0134d80d26a30)
    sk_bob = 0x13803677559c049a98a1576ad2f1fd15bdd9f81886f225c0850e883bd7d4cf63
    PK_bob = get_PK(sk_bob)
    PK_alice_fhe = get_PK(sk_alice_fhe)
    C2 = circuit.encrypt(sk_bob)
    PK_fhe_alice = secp256k1.add(PK_alice_fhe, PK_bob)
    HPK_fhe_alice = sha3.keccak_256(PK_fhe_alice[0].to_bytes(32,'big')+PK_fhe_alice[1].to_bytes(32,'big')).hexdigest()
    WA_fhe_alice = "0x"+ HPK_fhe_alice[-40:]
    C1 =  circuit.encrypt(sk_alice_fhe)
    C = C1 + C2
    sk_fhe = circuit.decrypt(C)
    PK_fhe_bob = get_PK(sk_fhe)
    HPK_fhe_bob = sha3.keccak_256(PK_fhe_bob[0].to_bytes(32,'big')+PK_fhe_bob[1].to_bytes(32,'big')).hexdigest()
    WA_fhe_bob = "0x"+ HPK_fhe_bob[-40:]
    end = time.perf_counter()
    elapsed_time = end - start
    print(elapsed_time)
    i+1
